In [0]:
from google.colab import drive
drive.mount('/content/gdrive' )
%cd '/content/gdrive/My Drive/Comp4107 Project'


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/Comp4107 Project


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  
import tensorflow as tf

In [0]:
from tensorflow.python.keras.optimizers import Adam, RMSprop
from tensorflow.python.keras.layers import Input, Dense,Lambda, Embedding, Flatten, Dropout, merge, Activation, BatchNormalization, LeakyReLU
from tensorflow.python.keras.models import Model,Sequential
from tensorflow.python.keras.regularizers import l2
from tensorflow.python.keras import backend as K
from tensorflow.python.keras import regularizers
from tensorflow.python.keras import initializers
from tensorflow.python.keras.layers import add, concatenate

In [0]:
#https://stackoverflow.com/questions/44666098/how-to-implement-sparse-mean-squared-error-loss-in-keras MMSE
def rmse(y_true, y_pred):
    mask_true = K.cast(K.not_equal(y_true, 0), K.floatx())
    masked_squared_error = K.square(mask_true * (y_true - y_pred))
    masked_mse = K.sum(masked_squared_error, axis=-1) / K.sum(mask_true, axis=-1)
    return K.sqrt(masked_mse)

In [0]:
train_X = np.load("ML-1M/train1.npy")
test_X = np.load("ML-1M/test1.npy")

In [0]:
def run_experiments(models,title, x_train=train_X, x_test=test_X, epochs=150,start=5):
  plt.figure(figsize=(10,8))
  iters = np.arange(start,epochs)
  for i in range(len(models)):
    model = models[i][0]
    model.compile(optimizer='rmsprop', loss=rmse)
    label = models[i][1]
    losses = model.fit(x_train, x_train, 
                epochs=epochs,
                batch_size=256,
                shuffle=True,
                validation_data=(x_train , x_test))
    val_loss= np.array(losses.history["val_loss"])
    plt.plot(iters,val_loss[start:],label=label)
  
  plt.legend()
  # plt.ylim(top=1.5)
  plt.title(title,fontsize=20)
  plt.ylabel("Validation Loss")
  plt.xlabel("Epochs")
  plt.savefig(title+".png")

  plt.show()


####  One Layer AutoEncoder


In [0]:
def basicAE(X,encode_dim):
    input_user = Input(shape=(X.shape[1],))
    encoded = Dense(encode_dim, activation='elu')(input_user)
    decoded = Dense(X.shape[1], activation='elu')(encoded)

    return Model(input_user, decoded)

In [0]:
# we are masking some of the input ratings to add noise so hopefully
# our autoender can improve generalizabity 
def denoising_AE(X,encode_dim,dropout):
    input_user = Input(shape=(X.shape[1],))
    noisy_input = Dropout(rate=dropout)(input_user)
    encoded = Dense(encode_dim, activation='elu')(noisy_input)
    decoded = Dense(X.shape[1], activation='elu')(encoded)
    return Model(input_user, decoded)

In [0]:

experiments = [(basicAE(train_X,350),"Latent Space 350"),(basicAE(train_X,100),"Latent Space 100"),(basicAE(train_X,200),"Latent Space 200"),
                (basicAE(train_X,500),"Latent Space 500"),
                (basicAE(train_X,650),"Latent Space 650")]

run_experiments(models=experiments,title="Latent Space Dimension of Shallow Autoencoder",x_train=train_X,x_test=test_X,
                epochs=200,start=5) 

In [0]:
experiments = [(denoising_AE(train_X,350,0.1),"0.1"),(denoising_AE(train_X,350,0.2),"0.2"),
               (denoising_AE(train_X,350,0.35),"0.35"), (denoising_AE(train_X,350,0.5),"0.5"), (denoising_AE(train_X,350,0),"No dropout")]

run_experiments(models=experiments,title="Dropout Rate of Denoising Autoencoder",x_train=train_X,x_test=test_X,
                epochs=200,start=5)  

#### Deeper Autoencoders





In [0]:
def deepAE(X,layers,middle_dim,dropout):
    model = Sequential()
    #encode
    for i in range(len(layers)):
      if i ==0:
        model.add(Dense(layers[i],input_shape=(X.shape[1],), activation='selu'))
      else:
        model.add(Dense(layers[i], activation='selu'))
    
    #latent layer
    model.add(Dense(middle_dim, activation='selu'))
    model.add(Dropout(dropout))
    layers.reverse()
    #decode
    for i in range(len(layers)):
      model.add(Dense(layers[i], activation='selu'))

    #output
    model.add(Dense(X.shape[1], activation='selu'))
    return model

In [0]:
experiments = [(deepAE(train_X,[600,400],middle_dim=350,dropout=0.5),"Layers:600,400,350"),
               (deepAE(train_X,[500],middle_dim=350,dropout=0.5),"Layers:500,350"),
               (deepAE(train_X,[500,700],middle_dim=350,dropout=0.5),"Layers:500,700,350"),
               (deepAE(train_X,[300,500],middle_dim=750,dropout=0.5),"Layers:300,500,750"),
               (deepAE(train_X,[200,400],middle_dim=600,dropout=0.5),"Layers:200,400,600"),
               (deepAE(train_X,[128,256],middle_dim=256,dropout=0.5),"Layers:128,256,256"),
               (deepAE(train_X,[128,256,256],middle_dim=256,dropout=0.5),"Layers:128,256,256,256")]



run_experiments(models=experiments,title="Dimension sizes of Deeper Autoencoders",x_train=train_X,x_test=test_X,
                epochs=150,start=5)   

In [0]:
def deepAE_refeeding(X,layers,middle_dim,dropout):
    weights=[]
    #encode
    for i in range(len(layers)):
        weights.append(Dense(layers[i], activation='selu'))
    #middle
    weights.append(Dense(middle_dim, activation='selu' ))
    weights.append(Dropout(dropout))
    #decode
    layers.reverse()
    for i in range(len(layers)):
        weights.append(Dense(layers[i], activation='selu' ))
    weights.append(Dense(X.shape[1], activation='selu' )) #output
    
    inputX = Input(shape=(X.shape[1],))
    x_feed1 = K.identity(inputX)
    for i in range(len(weights)):
        x_feed1 = weights[i](x_feed1)
    output1 = x_feed2 = x_feed1
    for i in range(len(weights)):
        x_feed2 = weights[i](x_feed2)
    output2 = x_feed2
    model = Model(inputs=inputX, outputs=output2) 
    return model



In [0]:
experiments = [(deepAE_refeeding(train_X,[500],350,0.5),"Refeed - Layers:500,350"),
               (deepAE_refeeding(train_X,[700,500],350,0.5),"Refeed - Layers:700,500,350"),
               (deepAE_refeeding(train_X,[500],700,0.5),"Refeed - Layers:500,700"),
               (deepAE(train_X,[500],350,0.5),"No refeeding - Layers:500,350"),] 

run_experiments(models=experiments,title="Dense Refeeding Deep Autoencoder",x_train=train_X,x_test=test_X,
                epochs=200,start=5)   

In [0]:
#batch normalized
def deepAE_BN(X,layers,middle_dim,dropout):
    model = Sequential()
    #encode
    for i in range(len(layers)):
      if i ==0:
        model.add(Dense(layers[i],input_shape=(X.shape[1],), activation='selu' ))
      else:
        model.add(Dense(layers[i], activation='selu' ))
      model.add(BatchNormalization())
    #latent layer
    model.add(Dense(middle_dim, activation='selu' ))
    model.add(Dropout(dropout))
    layers.reverse()
    #decode
    for i in range(len(layers)):
      model.add(Dense(layers[i], activation='selu' ))
      model.add(BatchNormalization())
    #output
    model.add(Dense(X.shape[1], activation='selu' ))
    return model


In [0]:

experiments = [(deepAE_BN(train_X,[128,256],256,0.5),"BN - Layers:128,256,256"),
               (deepAE_BN(train_X,[500],350,0.5),"BN - Layers:500,350"),
               (deepAE_BN(train_X,[700,500],350,0.5),"BN - Layers:700,500,350"),
               (deepAE_BN(train_X,[500],700,0.5),"BN - Layers:500,700"),
               (deepAE(train_X,[500],350,0.5),"No BN - Layers:500,350"),]

run_experiments(models=experiments,title="Batch Normalized Deep Autoencoder",x_train=train_X,x_test=test_X,
                epochs=200,start=5)   

In [0]:
import time
def run_results(ae,args):
    final_loss=[]
    times =[]
    for i in range(1,6):
        model = ae(*args)
        model.summary()
        train_X_fold = np.load("ML-1M/train"+str(i)+".npy")
        test_X_fold = np.load("ML-1M/test"+str(i)+".npy")
        model.compile( optimizer='rmsprop', loss=rmse)
        t1 = time.process_time()
        losses = model.fit(train_X_fold, train_X_fold,
                        epochs=300, 
                        batch_size=256,
                        shuffle=True,
                        validation_data=(train_X_fold , test_X_fold)) 
        t2 = time.process_time()
        final_loss.append(losses.history["val_loss"][299])
        del model #memory saving
        times.append(t2-t1)
    return np.array(final_loss),np.array(times)


In [0]:
args=[train_X,500]
loos,tims = run_results( basicAE,args)
np.save("basic_ae_loss.npy",loos)
np.save("basic_ae_times.npy",tims)   


In [0]:
print(loos.mean(), 2*loos.std()/5**0.5)
print(tims.mean(), 2*tims.std()/5**0.5)
loos

0.18296211063861847 0.003033946559135974
1557.201540491201 11.421088213593924


array([0.18089859, 0.18527307, 0.18153077, 0.17880106, 0.18830706])